## Create Playlist
By:- Vineet Singh Negi

This notebook uses Spotipy to create playlists for the song data collected previously programmatically.

In [1]:
import pandas as pd
import numpy as np
import math

import spotipy
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials

from tqdm import tqdm

Note: Ensure you have already set your environment variables and your Spotify app before executing the code in this notebook. 

The environment variables that should be set:

-> SPOTIPY_CLIENT_ID - Get this from your Spotify app in the Developer Console

-> SPOTIPY_CLIENT_SECRET - Get this from your Spotify app in the Developer Console

-> SPOTIPY_REDIRECT_URI - Set to http://localhost:8080

In [2]:
##==============================================
## Function to get Client Id and Client Secret
##==============================================

client_id = ''  # <-- Enter your client it 
client_secret = '' # <-- Enter your client secret

auth_manager = SpotifyOAuth(client_id = client_id, redirect_uri='http://localhost:8080',
                            client_secret = client_secret, scope="playlist-modify-public", 
                            open_browser=False)  

In [3]:
##===================
## Set OAuth Token
##===================

auth_manager.get_authorize_url()

'https://accounts.spotify.com/authorize?client_id=9024429ddca6460c8b6fc35343a37b00&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%3A8080&scope=playlist-modify-public'

In [ ]:
auth_manager.get_access_token('', as_dict=False) # <-- Enter token

In [5]:
##=================================
## Authorize and get current user
##=================================

spotify = spotipy.Spotify(auth_manager=auth_manager)

user_dict = spotify.current_user()
user_dict

{'display_name': 'Vineet Singh Negi',
 'external_urls': {'spotify': 'https://open.spotify.com/user/31oqjlsf3jf3h2eea5h6pf2quu7m'},
 'href': 'https://api.spotify.com/v1/users/31oqjlsf3jf3h2eea5h6pf2quu7m',
 'id': '31oqjlsf3jf3h2eea5h6pf2quu7m',
 'images': [],
 'type': 'user',
 'uri': 'spotify:user:31oqjlsf3jf3h2eea5h6pf2quu7m',
 'followers': {'href': None, 'total': 0}}

In [6]:
df = pd.read_csv("songs.csv")
years = df['year'].unique()
years

array([1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968,
       1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979,
       1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990,
       1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001,
       2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012,
       2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022],
      dtype=int64)

In [7]:
StartYear = 1990
EndYear = 2010

#Some form validation to make sure the years make sense for the playlist.
if StartYear > EndYear:
    StartYear = EndYear

In [8]:
##=============================
## Creating Spotify Playlist
##=============================

pd.DataFrame(columns=['id','name','link']).to_csv("playlists.csv", index=False)

def check_playlists(playlist_name, playlists_df):
    """Checks if playlist has already been created."""
    result = playlists_df[playlists_df['name'] == playlist_name].shape[0]
    if result > 0:
        return False
    else:
        return True
    
def save_playlists(playlist, playlist_name, playlists_df):
    """Saves playlist in a csv file."""
    new_playlist = {
        'name':playlist_name,
        'id':playlist['id'],
        'link':playlist['external_urls']['spotify']
    }

    playlists_df = pd.concat([playlists_df, pd.DataFrame([new_playlist])])
    playlists_df.to_csv("playlists.csv", index=False)

def make_playlist(start_year, end_year):
    """Makes the playlist and adds tracks from songs.csv given the start and end year"""
    playlists_df = pd.read_csv("playlists.csv")
    if (start_year - end_year) == 0:
        playlist_name = f"Top US Singles: {start_year}"
    else:
        playlist_name = f"Top US Singles: {start_year}-{end_year}"

    if check_playlists(playlist_name, playlists_df):
        description = 'This playlist was generated automatically for a project on Datafantic.com.'
        playlist = spotify.user_playlist_create(user=user_dict['id'], 
                                                name=playlist_name,
                                                description=description)
        uris = list(df[(df['year'] >= start_year) & (df['year'] <= end_year)]['spotify_uri'].values)

        chunked_uris = np.array_split(uris, math.ceil(len(uris) / 100))
        for uri_list in chunked_uris:
            spotify.user_playlist_add_tracks(user=user_dict['id'], 
                                            playlist_id=playlist['id'], 
                                            tracks=uri_list)

        save_playlists(playlist, playlist_name, playlists_df)

In [9]:
##=============================
## Generating List of Tuples
##=============================

years = list(range(1958, 2023))

year_pairs = []
for width in range(0,21):
    for year in years[:-width]:
        year_pairs.append((year, year + width))

len(year_pairs)

1090

In [11]:
# Making playlists for single years.

for year in years:
    make_playlist(year, year)

In [ ]:
for pair in tqdm(year_pairs):
    make_playlist(pair[0], pair[1]) #This will take aound 40min or so to process.

In [17]:
df = pd.read_csv("playlists.csv")

In [18]:
df.shape

(1155, 3)